Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [68]:
#import
import pymongo
import requests
import lxml
from lxml import html
from bs4 import BeautifulSoup


#create connection
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.vacancy_hh
    
#parsing data
pages = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
query = input('Enter the title: ').replace(' ', '+')
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0'}
result_data = []

############################
# goes through all the vacancies on the page.
# at the next iteration, it goes to a new page.
############################
for page in pages:
    url = f'https://spb.hh.ru/search/vacancy?text={query}&page={page}'
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    all_div_vacancy = soup.find_all('div', class_='vacancy-serp-item-body__main-info')
    for div in all_div_vacancy:
        vacancy_data = []
        vacancy_data.append(div.find('a', class_='serp-item__title').get('href'))
        vacancy_data.append(div.find('a', class_='serp-item__title').contents[0])
        vacancy_data.append('https://spb.hh.ru/')
        try:
            vacancy_data.append(''.join(div.find('a', class_='bloko-link bloko-link_kind-tertiary').contents))
        except:
            vacancy_data.append(' ')
        try:
            vacancy_data.append(''.join(div.find('span', class_='bloko-header-section-3').contents))
        except:
            vacancy_data.append(' ')
        vacancy_data.append(str((div.find('a', class_='serp-item__title').get('href').split('/')[4]).split('?')[0]))
        result_data.append(vacancy_data)
    print(f'Page {page} - complete')
        
############################
# format salary
############################

def salary_format(str_salary):
    if str_salary == ' ':
        return [0, 0, ' ']
    elif ('от' in str_salary):
        salary_min = int(str(str_salary).replace('\u202f', '').split(' ')[2])
        salary_max = 0
        valute = str(str_salary).replace('\u202f', '').split(' ')[5]
        return salary_min, salary_max, valute
    elif ('до' in str_salary):
        salary_max = int(str(str_salary).replace('\u202f', '').split(' ')[2])
        salary_min = 0
        valute = str(str_salary).replace('\u202f', '').split(' ')[5]
        return salary_min, salary_max, valute
    else:
        salary_min = int(str_salary.replace('\u202f', '').split(' ')[0])
        salary_max = int(str_salary.replace('\u202f', '').split(' ')[2])
        valute = str(str_salary).replace('\u202f', '').split(' ')[4]
        return [salary_min, salary_max, valute]

############################
#check dublicate vacancy and insert data
############################
# structure collection:
#     id_url: id_url,
#     title: title,
#     salary: salary,
#     url: url,
#     company: company,
#     service: service
############################    
 
for el in result_data:
    if len(list(db.vacancy.find({'id_url': el[5]}))):
        continue
    else:
        db.vacancy.insert_one({ 
            'id_url': el[5],
            'title': el[1],
            'salary_min': salary_format(el[4])[0],
            'salary_max': salary_format(el[4])[1],
            'valute': salary_format(el[4])[2],
            'url': el[0],
            'company': el[3],
            'service': el[2]
        })

print('Complete.')

Enter the title: Анализ рисков
Page 1 - complete
Page 2 - complete
Page 3 - complete
Page 4 - complete
Page 5 - complete
Page 6 - complete
Page 7 - complete
Page 8 - complete
Page 9 - complete
Page 10 - complete
Complete.


2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).


In [73]:
salary = int(input("Enter the salary: "))

vacancy_list = db.vacancy.find({'salary_max': {'$gt': salary}})
for vacancy in vacancy_list:
    print(f'{vacancy["title"]}\n{vacancy["url"]}')

Enter the salary: 150000
Ведущий Python-developer
https://spb.hh.ru/vacancy/77013935?from=vacancy_search_list&query=python+developer
Python-developer
https://spb.hh.ru/vacancy/67635148?from=vacancy_search_list&query=python+developer
Python разработчик
https://spb.hh.ru/vacancy/71069892?from=vacancy_search_list&query=python+developer
Python Team Lead
https://spb.hh.ru/vacancy/77324513?from=vacancy_search_list&query=python+developer
Python разработчик
https://spb.hh.ru/vacancy/77319651?from=vacancy_search_list&query=python+developer
ML разработчик GPT
https://spb.hh.ru/vacancy/77327734?from=vacancy_search_list&query=python+developer
Middle Python developer
https://spb.hh.ru/vacancy/76595028?from=vacancy_search_list&query=python+developer
Middle Python Developer (Backend / Server side)
https://spb.hh.ru/vacancy/77148120?from=vacancy_search_list&query=python+developer
Senior Python developer (Django)
https://spb.hh.ru/vacancy/76821518?from=vacancy_search_list&query=python+developer
Python 